# Raw Data Processing

In [53]:
import os
import glob
import pathlib
import numpy as np
import pandas as pd
from numba import jit
import joblib
from joblib import Parallel, delayed
from rich.progress import track

from typing import Dict

import sys
sys.path.append("../src")

import config

from utils.common import timer
from utils.common import load_pickle, dump_pickle, save_cache
from utils.feature import FeatureStore

## Extract data each data_type from txt file

In [2]:
src_dir = pathlib.Path("../data/raw/train/")
filepaths = [
    path_filepath 
    for site_filepath in src_dir.glob("*") 
    for floor_filepath in site_filepath.glob("*") 
    for path_filepath in floor_filepath.glob("*")
]

In [3]:
filepath = filepaths[0]

site_id = filepath.parent.parent.name
floor = filepath.parent.name
path_id = filepath.name.split(".")[0]

feature = FeatureStore(
    site_id=site_id, 
    floor=floor, 
    path_id=path_id, 
    input_path="../data/raw/"
)
feature.load_all_data()

In [4]:
feature.save()

In [5]:
# attributes of site info
[c for c in dir(feature.site_info) if c[0]!="_"]

['draw_polygon',
 'features',
 'floor',
 'floor_coordinates',
 'get_site_info',
 'input_path',
 'map_type',
 'show_site_image',
 'site_height',
 'site_id',
 'site_width',
 'store_coordinates']

In [6]:
feature.site_info.site_height, feature.site_info.site_width

(212.06273086548222, 146.74902374734265)

In [7]:
feature.site_id, feature.floor, feature.path_id

('5da138764db8ce0c98bcaa46', 'F4', '5dabfad918410e00067e70ba')

In [8]:
# attributes of feature store
for d_type in feature.df_types:
    print(d_type)
    display(feature[d_type].head())

accelerometer


,timestamp,x,y,z,accuracy
0,1571551566693,-2.201767,0.078979,11.103226,2
1,1571551566713,-2.435196,-0.066452,9.657715,2
2,1571551566733,-2.476501,-0.052094,9.203415,2
3,1571551566753,-2.560898,-0.108368,9.472763,2
4,1571551566773,-2.646484,-0.138885,9.753494,2


accelerometer_uncalibrated


,timestamp,x,y,z,x2,y2,z2,accuracy
0,1571551566693,-2.028198,0.101730,11.459961,0.0,0.0,0.0,3
1,1571551566713,-2.363373,0.041275,10.371185,0.0,0.0,0.0,3
2,1571551566733,-2.490860,-0.076035,9.227966,0.0,0.0,0.0,3
3,1571551566753,-2.508224,-0.073044,9.332108,0.0,0.0,0.0,3
4,1571551566773,-2.547729,-0.161636,9.611633,0.0,0.0,0.0,3


beacon


...

,timestamp,x,y,z,accuracy
0,1571551566693,-0.033759,0.047460,0.686108,3
1,1571551566713,-0.038705,0.053961,0.685716,3
2,1571551566733,-0.045757,0.061953,0.686550,3
3,1571551566753,-0.053042,0.069828,0.688253,3
4,1571551566773,-0.060194,0.075623,0.690391,3


waypoint


,timestamp,x,y
0,1571551566576,37.889812,154.43535
1,1571551573569,27.694906,153.98010
2,1571551581118,16.998966,153.33621


wifi


,timestamp,ssid,bssid,rssi,frequency,last_seen_timestamp
0,1571551568450,64f1125cb0e5e507a7ba5c32dd76bd506f30ce94,d2b9915dc73e4d333a718f8c02edae5e2a4d94f5,-52,5785,1571551567798
1,1571551568450,b01857e452a84ace381ac545896264c61ac82a57,5db8a385607a001cae8da5f069e1005f527ae7d6,-53,2437,1571551556712
2,1571551568450,da39a3ee5e6b4b0d3255bfef95601890afd80709,4c1ab193093f7057e6678f8f12f7ac4c05b95680,-53,5785,1571551567800
3,1571551568450,da39a3ee5e6b4b0d3255bfef95601890afd80709,f20391acb21826bb8f38243de772b7f3f8301f83,-53,5785,1571551567800
4,1571551568450,da39a3ee5e6b4b0d3255bfef95601890afd80709,45708a1205fbe53ae5ced9e450e0cedccf96e05a,-54,2437,1571551566923


---

## Create map for label encode

In [80]:
src_dir = pathlib.Path("../data/raw/train/")
filepaths = [
    path_filepath 
    for site_filepath in src_dir.glob("*") 
    for floor_filepath in site_filepath.glob("*") 
    for path_filepath in floor_filepath.glob("*")
]

In [109]:
def get_bssid_from_featureStore(filepath):
    site_id = filepath.parent.parent.name
    floor = filepath.parent.name
    path_id = filepath.name.split(".")[0]
    
    feature = load_pickle(f"../data/working/{path_id}.pkl", verbose=False)
    uniques = feature.wifi.bssid.unique()
    if len(uniques) > 0:
        return uniques
    else:
        return np.array([])

In [110]:
%%time
bssid = Parallel(n_jobs=-1)(delayed(get_bssid_from_featureStore)(filepath) for filepath in track(filepaths))

bssid = np.concatenate(bssid, axis=0)
unique_bsid = np.unique(bssid)

bssid_map = {_bssid: i for i, _bssid in enumerate(bssid)}
dump_pickle("./tmp/map_bssid.pkl", bssid_map)

Output()

CPU times: user 16.4 s, sys: 3.01 s, total: 19.4 s
Wall time: 1min 33s


---

## Simple feature engineering

### Create waypoint

In [147]:
src_dir = pathlib.Path("../data/raw/train/")
filepaths = [
    path_filepath 
    for site_filepath in src_dir.glob("*") 
    for floor_filepath in site_filepath.glob("*") 
    for path_filepath in floor_filepath.glob("*")
]

In [164]:
%%time
def create_waypoint(filepaths: List):
    def get_waypoint_from_featureStore(filepath):
        path_id = filepath.name.split(".")[0]

        feature = load_pickle(f"../data/working/{path_id}.pkl", verbose=False)
        wp = feature['waypoint']
        wp['site'] = feature.site_id
        wp['floor'] = feature.n_floor
        wp['path'] = feature.path_id
        if len(wp) > 0:
            return wp
        else:
            return pd.DataFrame([])

    waypoint = Parallel(n_jobs=-1)(delayed(get_waypoint_from_featureStore)(filepath) for filepath in track(filepaths))
    waypoint = pd.concat(waypoint, axis=0).reset_index(drop=True)
    return waypoint


waypoint = create_waypoint(filepaths)

Output()

CPU times: user 1min 11s, sys: 3.12 s, total: 1min 14s
Wall time: 1min 47s


In [165]:
waypoint.head()

,timestamp,x,y,site,floor,path
0,1571551566576,37.889812,154.43535,5da138764db8ce0c98bcaa46,3,5dabfad918410e00067e70ba
1,1571551573569,27.694906,153.98010,5da138764db8ce0c98bcaa46,3,5dabfad918410e00067e70ba
2,1571551581118,16.998966,153.33621,5da138764db8ce0c98bcaa46,3,5dabfad918410e00067e70ba
3,1571568619480,36.867283,179.88359,5da138764db8ce0c98bcaa46,3,5dac3de918410e00067e7244
4,1571568624436,37.372460,185.84445,5da138764db8ce0c98bcaa46,3,5dac3de918410e00067e7244


### Create wifi 

In [112]:
filepath = filepaths[0]

site_id = filepath.parent.parent.name
floor = filepath.parent.name
path_id = filepath.name.split(".")[0]

feature = load_pickle(f"../data/working/{path_id}.pkl")

Load pickle from ../data/working/5dabfad918410e00067e70ba.pkl


In [116]:
feature['waypoint']

,timestamp,x,y
0,1571551566576,37.889812,154.43535
1,1571551573569,27.694906,153.98010
2,1571551581118,16.998966,153.33621


In [113]:
# wifi data processing

wp = feature['waypoint']
wifi = feature['wifi']

In [114]:
for i in range(len(wp)):
    ts_pre_wp = wp.loc[i - 1, 'timestamp'] if i > 0 else None
    ts_current_wp = wp.loc[i, 'timestamp']
    ts_post_wp = wp.loc[i + 1, 'timestamp'] if (i + 1) < len(wp) else None

    _wifi = feature.wifi.copy()
    # NOTE: ターゲットとなるwaypointとその前後のwaypointの間にあるデータを取得する。
    _ts_wifi = _wifi['timestamp'].values
    pre_flag = np.ones(len(_ts_wifi)).astype(bool) if ts_pre_wp==None else (ts_pre_wp < _ts_wifi)
    psot_flag = np.ones(len(_ts_wifi)).astype(bool) if ts_post_wp==None else (_ts_wifi < ts_post_wp)
    _wifi = _wifi[pre_flag & psot_flag]
    
    _wifi['bssid'] = _wifi['bssid'].map(bssid_map)
    
    _wifi = _wifi.sort_values(by='rssi', ascending=False)
    _wifi = _wifi.head(100)
    
    wifi_feature = {
        'bssid': _wifi['bssid'].astype('int32').to_numpy(),
        'rssi': _wifi['rssi'].astype('float32').to_numpy(),
        'frequency': _wifi['frequency'].astype('float32').to_numpy(),
    }
    
    print(wifi_feature)

{'bssid': array([58892, 58892, 52822, 52822, 52799, 52799, 32558, 52821, 52821,
       52799, 32590, 32590, 32558, 58866, 58866, 58866, 32551, 58919,
       58919, 58892, 47899, 32558, 59810, 32590, 47899, 53624, 58837,
       32585, 55603, 55614, 55583, 55583, 55583, 55603, 59810, 32551,
       55603, 59810, 59734, 47899, 59734, 47900, 59734, 58837, 59527,
       55614, 59527, 55614, 59527, 19585, 19585, 59517, 59528, 21078,
       59517, 59528, 19585, 21078, 58932, 21078, 59528, 59517, 11398,
       59529, 11398, 47900, 58937, 59529, 47900, 11398, 58938, 11330,
       59534, 11330, 59512, 11330, 59512, 59534, 32551, 32626, 53624,
       58919, 32638, 32638, 32638, 59534, 59529, 52821, 52822, 59512,
       53624, 58932, 52791, 52791, 58932, 58937, 58938, 58938, 58937,
       32660], dtype=int32), 'rssi': array([-48., -48., -52., -52., -53., -53., -53., -53., -53., -53., -54.,
       -54., -55., -56., -57., -57., -58., -59., -59., -59., -61., -62.,
       -62., -63., -63., -64., -64., 